In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 5.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import optuna
import random
from optuna.samplers import TPESampler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, StackingClassifier, VotingClassifier

random.seed(42)

In [ ]:
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_val = pd.read_csv('X_val.csv')
y_val = pd.read_csv('y_val.csv')

In [ ]:
# Baseline

model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
score = roc_auc_score(y_val, y_pred)
print(score)

0.6665318410589938


In [ ]:
def objective(trial):
    param_grid = {
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'splitter': trial.suggest_categorical('splitter', ['best', 'random']),
        'max_depth': trial.suggest_int('max_depth', 1, 30),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 50),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 50),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
        'min_weight_fraction_leaf': trial.suggest_float('min_weight_fraction_leaf', 0.0, 0.5),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 2, 50),
        'ccp_alpha': trial.suggest_float("ccp_alpha", 1e-4, 0.1, log=True),
        'class_weight': trial.suggest_categorical('class_weight', ['balanced', None])
    }
    model = DecisionTreeClassifier(**param_grid, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    score = roc_auc_score(y_val, y_pred)
    return score

sampler = TPESampler(seed=42)
study = optuna.create_study(direction='maximize', sampler=sampler)
study.enqueue_trial({'criterion': 'entropy', 'splitter': 'random', 'max_depth': 17, 'min_samples_split': 7, 'min_samples_leaf': 20, 'max_features': None, 'min_weight_fraction_leaf': 0.06155467087603367, 'max_leaf_nodes': 46, 'ccp_alpha': 0.0005113352284108883, 'class_weight': 'balanced'})
study.optimize(objective, n_trials=2500)

[I 2025-03-06 12:50:56,942] A new study created in memory with name: no-name-0928da41-15de-4807-859a-27d9a12241a6
[I 2025-03-06 12:50:56,960] Trial 0 finished with value: 0.6767885459517845 and parameters: {'criterion': 'entropy', 'splitter': 'random', 'max_depth': 17, 'min_samples_split': 7, 'min_samples_leaf': 20, 'max_features': None, 'min_weight_fraction_leaf': 0.06155467087603367, 'max_leaf_nodes': 46, 'ccp_alpha': 0.0005113352284108883, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.6767885459517845.
[I 2025-03-06 12:50:56,978] Trial 1 finished with value: 0.5693126192172454 and parameters: {'criterion': 'entropy', 'splitter': 'best', 'max_depth': 5, 'min_samples_split': 9, 'min_samples_leaf': 3, 'max_features': 'sqrt', 'min_weight_fraction_leaf': 0.010292247147901223, 'max_leaf_nodes': 49, 'ccp_alpha': 0.03142880890840111, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.6767885459517845.
[I 2025-03-06 12:50:56,999] Trial 2 finished with value: 0.5669248270

In [ ]:
param_grid = study.best_params
model = DecisionTreeClassifier(**param_grid, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
score = roc_auc_score(y_val, y_pred)
print(score)

0.7158504355020338


In [ ]:
param_grid = study.best_params
model = DecisionTreeClassifier(**param_grid, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
score = roc_auc_score(y_train, y_pred)
print(score)

0.707844696969697


In [ ]:
unique_scores = sorted(set(t.value for t in study.trials if t.value is not None), reverse=True)[:50]
best_trial = next(t for t in study.trials if t.value == unique_scores[0])
models = [None] * 11
models[0] = DecisionTreeClassifier(**best_trial.params, random_state=42)

random.shuffle(unique_scores)
unique_scores = unique_scores[:10]
top_trials = [next(t for t in study.trials if t.value == v) for v in unique_scores]

for i in range(10):
    print(top_trials[i].value)
    params = top_trials[i].params
    models[i + 1] = DecisionTreeClassifier(**params, random_state=42)

0.6930779307333441
0.6937719761910234
0.6939259531634223
0.697639785811137
0.7045917312067658
0.687286558040126
0.6916094040861351
0.6979615287385379
0.6904626203663273
0.6945947188196632


In [ ]:
best_estimators = []
best_score = 0

for i in range(1, 1024):
    estimators = []
    for j in range(1, 11):
        x = 2 ** j
        if i % x >= x / 2:
            estimators.append(('clf' + str(j), models[j]))
    model = StackingClassifier(estimators=estimators, final_estimator=models[0], cv=5)
    model.fit(X_train, y_train)
    pred = model.predict(X_val)
    ids = []
    for id, model in estimators:
        ids.append(id)
    print(ids)
    print(roc_auc_score(y_val, pred))
    if roc_auc_score(y_val, pred) > best_score:
        best_estimators = ids
        best_score = roc_auc_score(y_val, pred)
print()
print(best_estimators)
print(best_score)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the 

['clf1']
0.6973410245214074
['clf2']
0.6376692023073564
['clf1', 'clf2']
0.6609656884149565
['clf3']
0.666364075103992
['clf1', 'clf3']
0.6398294762484775


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the 

['clf2', 'clf3']
0.6974490382184635
['clf1', 'clf2', 'clf3']
0.6303840231654907


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the 

...

['clf1', 'clf3', 'clf4', 'clf5', 'clf6', 'clf7', 'clf8', 'clf9', 'clf10']
0.6902833635924895


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


['clf2', 'clf3', 'clf4', 'clf5', 'clf6', 'clf7', 'clf8', 'clf9', 'clf10']
0.6705076643761635


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


['clf1', 'clf2', 'clf3', 'clf4', 'clf5', 'clf6', 'clf7', 'clf8', 'clf9', 'clf10']
0.6080045044009835

['clf3', 'clf4', 'clf5', 'clf7', 'clf10']
0.733895617401696


In [ ]:
best_estimators = []
best_score = 0

for i in range(1, 2048):
    estimators = []
    for j in range(1, 12):
        x = 2 ** j
        if i % x >= x / 2:
            estimators.append(('clf' + str(j - 1), models[j - 1]))
    model = VotingClassifier(estimators=estimators, voting='soft')
    model.fit(X_train, y_train)
    pred = model.predict(X_val)
    ids = []
    for id, model in estimators:
        ids.append(id)
    print(ids)
    print(roc_auc_score(y_val, pred))
    if roc_auc_score(y_val, pred) > best_score:
        best_estimators = ids
        best_score = roc_auc_score(y_val, pred)
print()
print(best_estimators)
print(best_score)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the 

['clf0']
0.7158504355020338
['clf1']
0.6930779307333441
['clf0', 'clf1']
0.7135936386826925
['clf2']
0.6937719761910234
['clf0', 'clf2']
0.7064647346769931
['clf1', 'clf2']
0.7034794199434653
['clf0', 'clf1', 'clf2']
0.7101188150667617
['clf3']
0.6939259531634223
['clf0', 'clf3']
0.7020821363730378
['clf1', 'clf3']
0.7036104152781928
['clf0', 'clf1', 'clf3']
0.7023923884816032
['clf2', 'clf3']
0.7082917748718773
['clf0', 'clf2', 'clf3']
0.6969710201548962
['clf1', 'clf2', 'clf3']


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the 

0.7046974467400547
['clf0', 'clf1', 'clf2', 'clf3']
0.7018431273412543
['clf4']
0.697639785811137
['clf0', 'clf4']
0.7146209178866086
['clf1', 'clf4']
0.7129846252843978
['clf0', 'clf1', 'clf4']
0.7161491967917635


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the 

...

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the 

['clf2', 'clf3', 'clf4', 'clf5', 'clf6', 'clf7', 'clf8', 'clf9', 'clf10']
0.7206398087927746
['clf0', 'clf2', 'clf3', 'clf4', 'clf5', 'clf6', 'clf7', 'clf8', 'clf9', 'clf10']
0.7206398087927746


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the 

['clf1', 'clf2', 'clf3', 'clf4', 'clf5', 'clf6', 'clf7', 'clf8', 'clf9', 'clf10']
0.7218578355893641
['clf0', 'clf1', 'clf2', 'clf3', 'clf4', 'clf5', 'clf6', 'clf7', 'clf8', 'clf9', 'clf10']
0.7205800565348287

['clf0', 'clf2', 'clf5', 'clf9', 'clf10']
0.738887229103946


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [ ]:
max_score = 0
best_iter = 0

for i in range(1, 250):
    model = BaggingClassifier(estimator=models[0], n_estimators=i, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    score = roc_auc_score(y_val, y_pred)
    if score > max_score:
        max_score = score
        best_iter = i
    print(i, score)

print(max_score, best_iter)

/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_bagging.py:878: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_bagging.py:878: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_bagging.py:878: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_bagging.py:878: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

1 0.5538873440121342
2 0.6505182359294924
3 0.6510077448118954
4 0.6766460597982211
5 0.6829752028129524
6 0.6893043458276837


/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_bagging.py:878: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_bagging.py:878: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_bagging.py:878: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_bagging.py:878: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

7 0.6899731114839244
8 0.6986188035759429
9 0.70500769884862
10 0.7049479465906741


/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_bagging.py:878: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_bagging.py:878: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_bagging.py:878: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

247 0.7068944913014501


/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_bagging.py:878: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


248 0.7062854779031554


/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_bagging.py:878: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


249 0.7050674511065659
0.7124353641440491 11


In [ ]:
def objective(trial):
    param_grid = {
        'n_estimators': trial.suggest_int('n_estimators', 1, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.1, 2.0, step=0.1)
    }
    model = AdaBoostClassifier(**param_grid, estimator=models[0], random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    score = roc_auc_score(y_val, y_pred)
    return score

sampler = TPESampler(seed=42)
study4 = optuna.create_study(direction='maximize', sampler=sampler)
study4.optimize(objective, n_trials=500)

[I 2025-03-06 13:26:54,901] A new study created in memory with name: no-name-85edc343-cb4f-48cf-ba1b-0333ed5abd8f
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2025-03-06 13:26:55,237] Trial 0 finished with value: 0.7094385585916853 and parameters: {'n_estimators': 38, 'learning_rate': 2.0}. Best is trial 0 with value: 0.7094385585916853.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2025-03-06 13:26:55,899] Trial 1 finished with value: 0.7656355571898055 and parameters: {'n_estimators': 74, 'learning_rate': 1.2000000000000002}. Best is trial 1 with valu